In [1]:
import json
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [3]:
data_one = []
reviewdata_df = []

In [4]:
data_file = open('F:\yelp\yelp_academic_dataset_review.json')
length = 2225213  # I calculated length prior to this calculation
for i, line in enumerate(data_file):
    if i<(length/4):
        data_one.append(json.loads(line))
data_file.close()

In [5]:
df = pd.DataFrame(data_one)

In [6]:
data_main = []
with open('F:\yelp\yelp_academic_dataset_user.json') as f:
    for line in f:
        data_main.append(json.loads(line))

In [7]:
df_user = pd.DataFrame(data_main)

In [8]:
def user_with_friends(x):
    if len(x['friends']) != 0:
        return x

In [9]:
df_user_with_friends = df_user.apply(user_with_friends, axis=1)

In [10]:
df_user_with_friends = df_user_with_friends.dropna()

In [11]:
merged_df = pd.merge(df, df_user_with_friends, on='user_id', how='inner')

In [12]:
merged_df.shape

(394815, 18)

In [13]:
data_business = []
with open('F:\yelp\yelp_academic_dataset_business.json') as f:
    for line in f:
        data_business.append(json.loads(line))

In [14]:
df_business = pd.DataFrame(data_business)

In [15]:
business_merged_df = pd.merge(merged_df, df_business, on='business_id', how='inner')

In [16]:
business_merged_df.drop(['neighborhoods', 'latitude', 'longitude', 'elite', 'yelping_since', 'compliments', 'full_address', 'city', 'state',
                         'categories', 'open', 'hours', 'attributes'], axis=1, inplace=True)

In [17]:
%timeit business_sorted_df = business_merged_df.sort_index(by=['business_id'], ascending=[True])

1 loops, best of 3: 1.11 s per loop


F:\PythonIDE\lib\site-packages\ipykernel\__main__.py:257: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [18]:
grouped = business_merged_df.groupby(business_merged_df['business_id'])

In [19]:
def look_for_user_ratings1(group, biz_id):
    ur = []
    for row in group.itertuples():
        ur.append({
                "user_id": row[7],
                "user_rating": row[4],
            })
    return ur


In [20]:
business_user1 = {}
li = []
for business_id, group in grouped:
    business_user1.update({
        business_id: look_for_user_ratings1(group, business_id)
    })

In [21]:
##This is the 2nd hashmap

user_friends = {}

for row in df_user_with_friends.itertuples():
    user_id = row[9]
#    print user_id
    user_friends.update({
        user_id: row[5],
    })

In [22]:
def look_for_business_ratings(business_user_dict, business_id, user_ids):
    ratings = business_user_dict.get(business_id)
    sum_of_stars = 0
    count = 0
    business_reviewers = [li['user_id'] for li in ratings]
    for i, uid in enumerate(business_reviewers):
        for user in user_ids:
            if uid == user:
                count += 1
                sum_of_stars += ratings[i]['user_rating']
    avg = float(sum_of_stars/float(count))
    return avg

In [23]:
new_df = []
for row in business_merged_df.itertuples():
    business_id = row[1]
    user_id = row[7]
    ratings = business_user1.get(business_id)
    
    friends = user_friends.get(user_id)
    business_reviewers = [li['user_id'] for li in ratings]

    common_reviewers = list(set(friends) & set(business_reviewers))
    if len(common_reviewers) == 0:
        continue
    else:
        friends_avg = look_for_business_ratings(business_user1, business_id, common_reviewers)
        
    new_df.append({
            "Business_id": business_id,
            "User_id": user_id,
            "friends_avg_rating": friends_avg,
            "user_rating": row[4],
            "friends_who_rated": len(common_reviewers),
            "friends": len(friends),
            "no_of_reviewers": len(business_reviewers),
            "business_rating": row[18]
        })

final_df = pd.DataFrame(new_df)

In [24]:
 final_df.shape

(130449, 8)

In [25]:
corr_df = final_df[['friends_avg_rating', 'user_rating']]

In [26]:
corr_df.corr()

,friends_avg_rating,user_rating
friends_avg_rating,1.000000,0.357001
user_rating,0.357001,1.000000


In [27]:
final_df_1 = final_df[(final_df.business_rating >= 1) & (final_df.business_rating <= 2)]
final_df_2 = final_df[(final_df.user_rating == 2)]
final_df_3 = final_df[(final_df.user_rating == 3)]
final_df_4_5 = final_df[(final_df.business_rating >= 4) & (final_df.business_rating <= 5)]
final_df_4 = final_df[(final_df.business_rating == 4)]
final_df_5 = final_df[(final_df.user_rating == 5)]

In [28]:
final_df_5 = final_df[(final_df.business_rating == 5)]

In [29]:
corr_df_4 = final_df_4[['friends_avg_rating', 'user_rating']]

In [30]:
#this is the one making the most sense
plt.axhline(y=4)
plt.scatter(final_df_4.index, final_df_4.user_rating, c='b', label='user ratings')
plt.scatter(final_df_4.index, final_df_4.friends_avg_rating, c='r', label='friends_avg')

In [31]:
plt.show()

In [32]:
#this is the one making the most sense
plt.axhline(y=5)
plt.scatter(final_df_5.index, final_df_5.user_rating, c='b', label='user ratings')
plt.scatter(final_df_5.index, final_df_5.friends_avg_rating, c='r', label='friends_avg')

In [33]:
plt.show()

In [34]:
final_df_3 = final_df[(final_df.business_rating == 3)]

In [35]:
final_df_3.reset_index(drop=True)

,Business_id,User_id,business_rating,friends,friends_avg_rating,friends_who_rated,no_of_reviewers,user_rating
0,Ln34kwDUNZPLhVhUqauLWQ,0RNjwIoruU0aQ3yMGiZBdA,3.0,1,1.000000,1,9,1
1,Ln34kwDUNZPLhVhUqauLWQ,lMZD3OpQOb1VijGm2sbfbQ,3.0,5,1.000000,1,9,1
2,gClSqcwuLsgyZRCzhJc7Dg,JPPhyFE-UE453zA6K0TVgw,3.0,31,3.000000,1,7,5
3,gClSqcwuLsgyZRCzhJc7Dg,T_wjLgPOPXry7Bea4MzoVQ,3.0,79,3.000000,1,7,3
4,gClSqcwuLsgyZRCzhJc7Dg,T_wjLgPOPXry7Bea4MzoVQ,3.0,79,3.000000,1,7,1
5,gClSqcwuLsgyZRCzhJc7Dg,Q3fFv_ft17OyV-NRF1iQxw,3.0,59,3.000000,1,7,3
6,gClSqcwuLsgyZRCzhJc7Dg,UD7Y1CqfY6mDmRwIuCf6nA,3.0,955,3.200000,4,7,3
7,gClSqcwuLsgyZRCzhJc7Dg,nYKao-8w_IOj4Mv8QWb2vg,3.0,30,3.000000,1,7,4
8,eznh0XjVtv924K-KYx_ilQ,JPPhyFE-UE453zA6K0TVgw,3.0,31,3.333333,2,31,3
9,eznh0XjVtv924K-KYx_ilQ,bvu13GyOUwhEjPum2xjiqQ,3.0,230,3.454545,7,31,3


In [36]:
#this is the one making the most sense
plt.axhline(y=3)
plt.scatter(final_df_3.index, final_df_3.user_rating, c='b', label='user ratings')
plt.scatter(final_df_3.index, final_df_3.friends_avg_rating, c='r', label='friends_avg')

In [37]:
plt.show()

In [38]:
final_df

,Business_id,User_id,business_rating,friends,friends_avg_rating,friends_who_rated,no_of_reviewers,user_rating
0,mVHrayjG3uZ_RLHkLj-AMg,8fApIAMHn2MZJFUiCQto5Q,5.0,2,5.000000,1,9,5
1,mVHrayjG3uZ_RLHkLj-AMg,tAKjY3bQXH51msJbOHYPmQ,5.0,31,5.000000,1,9,5
2,YssP-_6iP0SLBsqUQGs6rQ,nEYPahVwXGD2Pjvgkm7QqQ,4.0,1444,4.000000,22,59,5
3,YssP-_6iP0SLBsqUQGs6rQ,GYNfjDbSlpdRK2qC8RqatA,4.0,45,4.000000,3,59,1
4,YssP-_6iP0SLBsqUQGs6rQ,r_96n9O0jNzbNCZDkSB0EQ,4.0,6,4.000000,1,59,5
5,YssP-_6iP0SLBsqUQGs6rQ,Enzdhh-rV5JgC4lgg0IdFw,4.0,149,3.333333,3,59,3
6,YssP-_6iP0SLBsqUQGs6rQ,ytgLwKzD6B4af5vW56RpJg,4.0,2,5.000000,1,59,5
7,YssP-_6iP0SLBsqUQGs6rQ,gtJJBaQwTwE45NnNF0coVA,4.0,64,4.333333,3,59,5
8,YssP-_6iP0SLBsqUQGs6rQ,vFgVosI6MUPX8Y23hmuMXw,4.0,5,5.000000,1,59,5
9,YssP-_6iP0SLBsqUQGs6rQ,RxEegiMllenlyhYgJqsTSQ,4.0,16,5.000000,1,59,4


In [39]:
final_df_no_duplicates = final_df.drop_duplicates(keep='first')

In [40]:
final_df_no_duplicates.shape

(127330, 8)

In [41]:
plt.scatter(final_df_no_duplicates.user_rating, final_df_no_duplicates.friends_avg_rating)

In [42]:
plt.show()

In [2]:
from sklearn.model_selection import train_test_split

In [51]:
final_df.size

1043592

In [61]:
train, test = train_test_split(final_df, test_size = 0.3)

In [62]:
train.size

730512

In [63]:
test.size

313080

In [64]:
X_train = train.user_rating
Y_train = train.friends_avg_rating

In [66]:
X_train.shape

(91314L,)

In [67]:
X_test = test.user_rating
Y_test = test.friends_avg_rating

In [68]:
X_test.shape

(39135L,)

In [70]:
X_train = X_train.reshape(len(X_train), 1)
Y_train = Y_train.reshape(len(Y_train), 1)

F:\PythonIDE\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
F:\PythonIDE\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  from ipykernel import kernelapp as app


In [71]:
clf = LinearRegression()
clf = clf.fit(X_train, Y_train)

In [74]:
X_test = X_test.reshape(len(X_test), 1)
Y_test = Y_test.reshape(len(Y_test), 1)

F:\PythonIDE\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
F:\PythonIDE\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  from ipykernel import kernelapp as app


In [75]:
r_sqr = clf.score(X_test, Y_test)
y_pred = clf.predict(X_test)

In [76]:
intercept = clf.intercept_
coefficient = clf.coef_

In [77]:
r_sqr

0.12950214593698417

In [78]:
y_pred

array([[ 3.84069456],
       [ 3.55877469],
       [ 3.84069456],
       ..., 
       [ 3.55877469],
       [ 4.12261442],
       [ 3.84069456]])

In [79]:
intercept

array([ 2.7130151])

In [80]:
coefficient

array([[ 0.28191986]])